In [28]:
from numpy import *

In [29]:
def loadDataSet(filename):
    dataMat = []
    fr = open(filename)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float, curLine))
        dataMat.append(fltLine)
    return dataMat

In [30]:
def distEclud(vecA, vecB):
    return sqrt(sum(power(vecA - vecB, 2)))

In [31]:
def randCent(dataset, k):
    n = shape(dataset)[1]
    centroids = mat(zeros((k, n)))
    for j in range(n):
        minJ = min(dataset[:, j])
        rangeJ = float(max(dataset[:, j]) - minJ)
        centroids[:, j] = minJ + rangeJ * random.rand(k, 1)
    return centroids

In [32]:
datMat = mat(loadDataSet('testSet.txt'))

In [33]:
min(datMat[:, 0])

matrix([[-5.379713]])

In [34]:
min(datMat[:, 1])

matrix([[-4.232586]])

In [35]:
max(datMat[:, 0])

matrix([[4.838138]])

In [36]:
max(datMat[:, 1])

matrix([[5.1904]])

In [37]:
randCent(datMat, 2)

matrix([[ 2.57737652, -1.90162389],
        [ 0.01797468, -0.99124161]])

In [38]:
distEclud(datMat[0], datMat[1])

5.184632816681332

In [45]:
def kMeans(dataset, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataset)[0]
    clusterAssment = mat(zeros((m, 2)))
    centroids = createCent(dataset, k)
    clusterChanged = True
    
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf
            minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j, :], dataset[i, :])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            if clusterAssment[i, 0] != minIndex:
                clusterChanged = True
            clusterAssment[i, :] = minIndex, minDist**2
        print(centroids)
        for cent in range(k):
            ptsInClust = dataset[nonzero(clusterAssment[:, 0].A == cent)[0]]
            centroids[cent, :] = mean(ptsInClust, axis=0)
    return centroids, clusterAssment

In [46]:
dataMat = mat(loadDataSet('testSet.txt'))

In [47]:
myCentroids, clustAssing = kMeans(dataMat, 4)

[[ 3.83987395 -3.30114478]
 [ 2.9688938   2.14504747]
 [ 1.73225712  0.56896469]
 [-3.37358249  3.67254536]]
[[ 2.51621757 -2.96890957]
 [ 2.6265299   3.10868015]
 [-2.88429492 -2.78125767]
 [-2.92772907  1.40343281]]
[[ 2.80293085 -2.7315146 ]
 [ 2.6265299   3.10868015]
 [-3.38237045 -2.9473363 ]
 [-2.46154315  2.78737555]]


In [52]:
def biKMeans(dataset, k, distMeas=distEclud):
    m = shape(dataset)[0]
    clusterAssment = mat(zeros((m, 2)))
    centroid0 = mean(dataset, axis=0).tolist()[0]
    centList = [centroid0]
    for j in range(m):
        clusterAssment[j, 1] = distMeas(mat(centroid0), dataset[j, :])**2
    while(len(centList) < k):
        lowestSSE = inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataset[nonzero(clusterAssment[:, 0].A == i)[0], :]
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)
            sseSplit = sum(splitClustAss[:, 1])
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:, 0].A != i)[0], 1])
            print("SSE split and notSplit:", sseSplit, sseNotSplit)
            if(sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[nonzero(bestClustAss[:, 0].A == 1)[0], 0] = len(centList)
        bestClustAss[nonzero(bestClustAss[:, 0].A == 0)[0], 0] = bestCentToSplit
        print("The best cent to split is:", bestCentToSplit)
        print("Size of best cent is:", len(bestClustAss))
        centList[bestCentToSplit] = bestNewCents[0, :]
        centList.append(bestNewCents[1, :])
        clusterAssment[nonzero(clusterAssment[:, 0].A == bestCentToSplit)[0], :] = bestClustAss
    return centList, clusterAssment

In [53]:
dataMat = mat(loadDataSet('testSet2.txt'))

In [54]:
centList, myNewAssments = biKMeans(dataMat, 3)

[[ 1.83307506 -4.10569911]
 [-4.51500369 -3.41485422]]
[[ 1.56386544  0.44157706]
 [-2.74010504  2.40095971]]
[[ 1.23710375  0.17480612]
 [-2.94737575  3.3263781 ]]
SSE split and notSplit: 570.7227574246755 0.0
The best cent to split is: 0
Size of best cent is: 60
[[-1.19694337 -2.27237651]
 [-0.74723577 -0.26632899]]
[[-0.43154563 -2.87788837]
 [ 2.74683414  2.93676781]]
[[-0.45965615 -2.7782156 ]
 [ 2.93386365  3.12782785]]
SSE split and notSplit: 68.68654812621844 38.06295063565756
[[-3.50024443  1.95849203]
 [-2.26808914  2.14603161]]
[[-3.70693073  3.29348345]
 [-2.01903078  3.36658267]]
[[-3.58362738  3.28784469]
 [-1.76576557  3.39794014]]
SSE split and notSplit: 22.971771896318412 532.6598067890178
The best cent to split is: 0
Size of best cent is: 40


In [55]:
centList

[matrix([[-0.45965615, -2.7782156 ]]),
 matrix([[-2.94737575,  3.3263781 ]]),
 matrix([[2.93386365, 3.12782785]])]